In [ ]:
import numpy as np
import cv2 as cv
import pandas
from glob import glob
import math
from tqdm import tqdm
from scipy.signal import triang

In [ ]:
train_img_paths = glob('train/*.jpeg')
train_img_names = [img_path.split('/')[-1].split('.')[0] for img_path in train_img_paths]
train_data = zip(train_img_paths, train_img_names)

In [ ]:
df = pandas.read_csv('trainLabels.csv')
data_dict = {}
for train_img_path ,train_img_name in train_data:
  row = df.loc[df['image'] == train_img_name]
  label = row.iloc[0]['level']
  data_dict[train_img_path] = label

print(data_dict)

In [ ]:
def separation_folder_hough(paths, out_dir):
    for path in paths:
        base_name = path.split('/')[1]
        img  = cv.imread(path)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
        circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 10, 10, minRadius = 900, maxRadius = 1700)
        circle  = circles[0][0]
        circle[2] *= 1.05
        
        min_x = np.clip(int(circle[0]-circle[2]), 0, img.shape[1]-1)
        max_x = np.clip(int(circle[0]+circle[2]), 0, img.shape[1]-1)
        min_y = np.clip(int(circle[1]-circle[2]), 0, img.shape[0]-1)
        max_y = np.clip(int(circle[1]+circle[2]), 0, img.shape[0]-1)
        
        img = img[min_y:max_y, min_x:max_x, ...]
        cv.imwrite(out_dir+'/'+base_name, img)    
        
        
def resize_folder(paths, out_dir, out_res):
    for path in paths:
        base_name = path.split('/')[1]
        img  = cv.imread(path)
       
        #preskalovani, aby vetsi strana byla rovna out_res
        if img.shape[0]>img.shape[1]:
            dx = int(out_res/img.shape[0] * img.shape[1])
            img = cv.resize(img, (dx, out_res), interpolation = cv.INTER_LANCZOS4)
        else:
            dy = int(out_res/img.shape[1] * img.shape[0])
            img = cv.resize(img, (out_res, dy), interpolation = cv.INTER_LANCZOS4)
            
        #vycentrovani    
        sy = (out_res-img.shape[0])//2
        sx = (out_res-img.shape[1])//2

        #vlozeni do ctvercoveho obrazku
        out_img = np.zeros((out_res, out_res, 3))
        out_img[sy:img.shape[0]+sy, sx:img.shape[1]+sx, ...] = img
        
        cv2.imwrite(out_dir+'/'+base_name, out_img)
def equalize(paths, out_dir):
  for path in paths:
    base_name = path.split('/')[1]
    img  = cv.imread(path)

    img = cv.cvtColor(img, cv.COLOR_BGR2LAB);
    clahe = cv.createCLAHE(clipLimit = 1.0, tileGridSize = (8,8))
    img[...,0] = clahe.apply(img[...,0])
    img = cv.cvtColor(img, cv.COLOR_LAB2BGR)
    cv.imwrite(out_dir+'/'+base_name, img)

In [ ]:
separation_folder_hough(train_img_paths, 'sample')
resize_folder(train_img_paths, 'sample', 1024)
equalize(train_img_paths, 'sample')            

avg_res  = [1024, 1024]
print(avg_res)

In [ ]:
factor = 1
len(data_dict.keys()) * (avg_res[0] / factor) * (avg_res[1] / factor) * 1 * 32 / 8 / 1024 / 1024 / 1024

In [ ]:
train_x = np.zeros((len(data_dict.keys()), math.ceil(avg_res[0] / factor) * math.ceil(avg_res[1] / factor)), dtype=np.float32)
train_y = np.zeros(len(data_dict.keys()), dtype=np.int32)

In [ ]:
tr = triang(factor*2+1).reshape(factor*2+1, 1)
kernel = np.dot(tr, tr.T)
kernel /= np.sum(kernel)

In [ ]:
i = 0
for img_path in tqdm(data_dict.keys()):
  img = cv.imread(img_path)
  img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  img = cv.resize(img, (avg_res[1], avg_res[0]))
  img = cv.filter2D(img, -1, kernel)[::factor, ::factor, ...]
  # feature extraction, student muze vlozit vlastni vektor features
  train_x[i, ...] = img.flatten() / 255.0
  train_y[i] = data_dict[img_path]
  i += 1

train_y[train_y != 0] = -1
train_y[train_y == 0] = 1

In [ ]:
svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

In [ ]:
svm.train(train_x, cv2.ml.ROW_SAMPLE, train_y)

In [ ]:
predictions = np.copy(train_y)
for i in tqdm(range(train_x.shape[0])):
  predictions[i] = svm.predict(train_x[i:i+1, ...])[1]

ind = predictions == train_y
print(np.sum(ind), np.sum(ind) / train_y.shape[0] * 100)